**474. Ones and Zeroes**

**Medium**

**Companies**: Google

You are given an array of binary strings strs and two integers m and n.

Return the size of the largest subset of strs such that there are at most m 0's and n 1's in the subset.

A set x is a subset of a set y if all elements of x are also elements of y.

**Example 1:**

```python
Input: strs = ["10","0001","111001","1","0"], m = 5, n = 3
Output: 4
```

**Explanation:** The largest subset with at most 5 0's and 3 1's is {"10", "0001", "1", "0"}, so the answer is 4.
Other valid but smaller subsets include {"0001", "1"} and {"10", "1", "0"}.
{"111001"} is an invalid subset because it contains 4 1's, greater than the maximum of 3.

**Example 2:**

```python
Input: strs = ["10","0","1"], m = 1, n = 1
Output: 2
```

**Explanation:** The largest subset is {"0", "1"}, so the answer is 2.

**Constraints**

- 1 <= strs.length <= 600
- 1 <= strs[i].length <= 100
- strs[i] consists only of digits '0' and '1'.
- 1 <= m, n <= 100


In [ ]:
# ----------------------------------------------------------
# Algorithm:
# 1. For each string, we have two choices: include it or exclude it.
# 2. Recursively explore all combinations while tracking the number of 0s and 1s used.
# 3. If we exceed the limits (m or n), return 0.
# 4. The maximum subset size is the maximum of:
#       - including the current string (if possible)
#       - excluding the current string
#
# Time Complexity: O(2^len(strs))  — every string has two choices
# Space Complexity: O(len(strs))   — recursion depth
# ----------------------------------------------------------

class Solution:
    def findMaxForm(self, strs: List[str], m: int, n: int) -> int:
        def count_zeros_ones(s):
            return s.count('0'), s.count('1')
        
        def dfs(index, zeros, ones):
            if index == len(strs):
                return 0
            
            z, o = count_zeros_ones(strs[index])
            
            # Option 1: skip current string
            not_take = dfs(index + 1, zeros, ones)
            
            # Option 2: take current string (if within limits)
            take = 0
            if zeros + z <= m and ones + o <= n:
                take = 1 + dfs(index + 1, zeros + z, ones + o)
            
            return max(take, not_take)
        
        return dfs(0, 0, 0)


In [ ]:
# ----------------------------------------------------------
# Algorithm:
# 1. Same logic as brute force, but store results in a memo table.
# 2. Memo key: (index, zeros, ones)
# 3. Each state represents the max subset size achievable starting
#    from `index` with `zeros` and `ones` used so far.
#
# Time Complexity: O(len(strs) * m * n)
# Space Complexity: O(len(strs) * m * n) due to memoization
# ----------------------------------------------------------

class Solution:
    def findMaxForm(self, strs: List[str], m: int, n: int) -> int:
        from functools import lru_cache
        
        counts = [(s.count('0'), s.count('1')) for s in strs]
        
        @lru_cache(None)
        def dfs(i, zeros, ones):
            if i == len(strs):
                return 0
            
            z, o = counts[i]
            
            not_take = dfs(i + 1, zeros, ones)
            take = 0
            if zeros + z <= m and ones + o <= n:
                take = 1 + dfs(i + 1, zeros + z, ones + o)
            
            return max(take, not_take)
        
        return dfs(0, 0, 0)


In [ ]:
# ----------------------------------------------------------
# Algorithm:
# 1. Treat each string as an item with "weight" = (zeros, ones)
#    and "value" = 1 (we want to maximize number of strings).
# 2. Use 2D DP table dp[m+1][n+1], where dp[i][j] is the maximum
#    subset size achievable using at most i zeros and j ones.
# 3. For each string (z, o), iterate backward from m → z, n → o:
#    dp[i][j] = max(dp[i][j], 1 + dp[i - z][j - o])
# 4. Return dp[m][n].
#
# Time Complexity: O(len(strs) * m * n)
# Space Complexity: O(m * n)
# ----------------------------------------------------------

class Solution:
    def findMaxForm(self, strs: List[str], m: int, n: int) -> int:
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        
        for s in strs:
            zeros = s.count('0')
            ones = s.count('1')
            
            for i in range(m, zeros - 1, -1):
                for j in range(n, ones - 1, -1):
                    dp[i][j] = max(dp[i][j], 1 + dp[i - zeros][j - ones])
        
        return dp[m][n]


In [ ]:
# ----------------------------------------------------------
# Algorithm: (avoid it please )
# 1. Create 3D DP table:
#       dp[i][j][k] = max subset size using first i strings,
#       with at most j zeros and k ones.
# 2. Transition:
#       - if not taking string i-1 → dp[i-1][j][k]
#       - if taking string i-1 → 1 + dp[i-1][j-z][k-o]
# 3. Take max of both choices.
# 4. Return dp[len(strs)][m][n].
#
# Time Complexity: O(N * m * n)
# Space Complexity: O(N * m * n)
# ----------------------------------------------------------

class Solution:
    def findMaxForm(self, strs: List[str], m: int, n: int) -> int:
        N = len(strs)
        dp = [[[0] * (n + 1) for _ in range(m + 1)] for _ in range(N + 1)]
        
        for i in range(1, N + 1):
            zeros = strs[i-1].count('0')
            ones = strs[i-1].count('1')
            
            for j in range(m + 1):
                for k in range(n + 1):
                    dp[i][j][k] = dp[i-1][j][k]
                    if j >= zeros and k >= ones:
                        dp[i][j][k] = max(dp[i][j][k], 1 + dp[i-1][j - zeros][k - ones])
        
        return dp[N][m][n]
